In [1]:
import pandas as pd

In [6]:
from langchain_community.document_loaders.csv_loader import CSVLoader


loader = CSVLoader(file_path=r"C:\Users\rohit\Downloads\Programs\project_data.csv.csv")
docs = loader.load()

In [7]:
docs[0]

Document(metadata={'source': 'C:\\Users\\rohit\\Downloads\\Programs\\flipcart_lapy_data.csv', 'row': 0}, page_content='clean_name: DELL Vostro Intel Core i3 12th Gen 1215U\nram_gb: 8.0\nprocessor: Intel Core i3\nos: Windows 11\nstorage_gb: 512.0\nDisplay Size: 15.6\ncpu_brand: Intel\nweight: 2.97\nbattery_life: 5.2\nprice: 42906\nRating: Not found')

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter


splitter= RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=400,
    chunk_overlap=50,
    
)
chunks = splitter.split_documents(docs)

In [9]:
print(chunks[0])

page_content='clean_name: DELL Vostro Intel Core i3 12th Gen 1215U
ram_gb: 8.0
processor: Intel Core i3
os: Windows 11
storage_gb: 512.0
Display Size: 15.6
cpu_brand: Intel
weight: 2.97
battery_life: 5.2
price: 42906
Rating: Not found' metadata={'source': 'C:\\Users\\rohit\\Downloads\\Programs\\flipcart_lapy_data.csv', 'row': 0}


In [10]:
len(chunks)

405

In [11]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS


embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
vectorstore = FAISS.from_documents(chunks, embedding_model)


C:\Users\rohit\AppData\Local\Temp\ipykernel_7312\1931618161.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
C:\Users\rohit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\rohit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packa

In [12]:
text = chunks[0].page_content
print("Text:\n", text)  

# Get embedding vector for this text
embedding_vector = embedding_model.embed_query(text)

# Show embedding
print("\nEmbedding vector:\n", embedding_vector)

# Optional: Just show first 10 numbers to keep it short
print("\nFirst 10 values:\n", embedding_vector[:10])


Text:
 clean_name: DELL Vostro Intel Core i3 12th Gen 1215U
ram_gb: 8.0
processor: Intel Core i3
os: Windows 11
storage_gb: 512.0
Display Size: 15.6
cpu_brand: Intel
weight: 2.97
battery_life: 5.2
price: 42906
Rating: Not found

Embedding vector:
 [0.004206650424748659, -0.04477153718471527, -0.03329736366868019, -0.024080492556095123, 0.05304640531539917, 0.007276966702193022, -0.010186169296503067, 0.0384075827896595, 0.0037796017713844776, 0.021179702132940292, -0.042825210839509964, -0.020074723288416862, 0.06284769624471664, 0.07243780046701431, -0.03363475576043129, 0.02828037180006504, 0.02820749580860138, 0.02495649643242359, 0.05109133571386337, 0.0140256118029356, -0.027566170319914818, -0.015269089490175247, 0.03617275133728981, -0.05679994821548462, -0.05151901766657829, -0.01715078204870224, 0.02734328992664814, -0.03674919158220291, 0.004967968445271254, -0.038408055901527405, 0.027146128937602043, -0.039526600390672684, -0.026141710579395294, 0.009205996058881283, 1.6946

In [13]:
query="i want a budget laptop for remote work with good batery life and i5"
result=vectorstore.similarity_search(query)
result[0].page_content

'clean_name: MSI Thin 15 Intel Core i5 12th Gen 12450H\nram_gb: 16.0\nprocessor: Intel Core i5\nos: Windows 11\nstorage_gb: 512.0\nDisplay Size: 15.6\ncpu_brand: Intel\nweight: 1.84\nbattery_life: 7.6\nprice: 54990\nRating: 4.4'

In [14]:
from langchain.chains import LLMChain, RetrievalQA
from langchain.chains import create_retrieval_chain
from langchain.prompts import PromptTemplate
from langchain.llms import Ollama
from langchain.prompts import ChatPromptTemplate
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


In [15]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})


In [16]:
llm = Ollama(model="llama3")

C:\Users\rohit\AppData\Local\Temp\ipykernel_7312\2786610211.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3")


In [17]:
# template = """Answer the question based only on the following context:

# {context}

# Question: {question}
# """
# prompt = ChatPromptTemplate.from_template(template)

template = """You are a laptop shopping assistant.
Understand the user request and rewrite it as a clear filter-based requirement for searching laptops.
Always display prices in Indian Rupees (₹), not in USD or any other currency.

User query: {query}
Refined requirement:"""


In [18]:
#prompt = PromptTemplate(input_variables=["question"], template=template)
#query_chain = LLMChain(llm=llm, prompt=prompt)

# Define the prompt template
prompt = PromptTemplate(input_variables=["question"], template=template)

# Compose the chain using the new Runnable pattern
query_chain = prompt | llm

# To run the chain
response = query_chain.invoke({"question": "What is the capital of France?"})


C:\Users\rohit\AppData\Local\Temp\ipykernel_7312\24208809.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  query_chain = LLMChain(llm=llm, prompt=prompt)


In [21]:
user_input = "I want a budget laptop for remote work"
refined_query = query_chain.run(user_input)
print("Refined Query:", refined_query)

Refined Query: Based on your query, here is a refined requirement that can be used to search for budget laptops suitable for remote work:

**Filter Requirements:**

1. **Price**: ₹30,000 - ₹50,000 (Budget-friendly options)
2. **Processor**: Intel Core i3 or equivalent (efficient for general tasks and remote work)
3. **RAM**: 4 GB or more (ensures smooth performance for multiple applications)
4. **Storage**: 256 GB or more (provides ample space for files and programs)
5. **Operating System**: Windows, Chrome OS, or Linux (popular options for remote work)
6. **Display**: HD or Full HD (1366x768 or 1920x1080) with a decent screen size (13-15 inches)
7. **Graphics**: Integrated graphics (Intel Iris or equivalent) or dedicated graphics (NVIDIA MX110 or higher)
8. **Connectivity**: Wi-Fi, Bluetooth, and USB ports (ensures seamless connectivity)

These filter requirements should help you find a budget-friendly laptop that meets your needs for remote work.


In [22]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})


In [23]:
rag_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)
response = rag_chain.run(refined_query)

In [24]:
print("\nRAG Response:\n", response)


RAG Response:
 Based on the provided context, I can answer the question as follows:

None of the laptops in the given context meet all the filter requirements. However, two laptops come close to meeting some of the requirements.

1. HP - The HP laptop has a price tag that is slightly above the budget range (₹46,390), but it meets the processor and RAM requirements.
2. MSI Thin 15 Intel Core i5 12th Gen 12450H - Two laptops with this model name come close to meeting some of the requirements. They have prices between ₹54,990 (above the budget range) and ₹51,090 (below the budget range), but they meet the processor, RAM, and storage requirements.

To find a laptop that meets all the filter requirements within the budget range of ₹30,000 - ₹50,000, further searching or exploring other options would be necessary.


In [25]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

C:\Users\rohit\AppData\Local\Temp\ipykernel_7312\1275336335.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [26]:
from langchain.agents import Tool
search_tool = Tool(
    name="LaptopSearch",
    func=lambda query: rag_chain.run(query),
    description="Useful for answering questions about laptop features, brands, budgets, etc."
)


from langchain.agents import initialize_agent, AgentType

agent_executor = initialize_agent(
    tools=[search_tool],
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True
)

C:\Users\rohit\AppData\Local\Temp\ipykernel_7312\3545154823.py:11: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_executor = initialize_agent(


In [27]:
response = agent_executor.run("Show me Dell laptops under ₹50000")
print(response)

# Later...
followup = agent_executor.run("Only i5 models please")
print(followup)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: LaptopSearch
Action Input: Show me Dell laptops under ₹50000
Observation: Based on the provided context, I can help you with that!

Here are the Dell laptops under ₹50000:

1. DELL Intel Core i5 12th Gen 1235U - Price: ₹44,990
2. DELL 15 Intel Core i5 13th Gen 1334U - Price: ₹49,750

These two laptops meet your budget of ₹50000.
Thought:Thought: Do I need to use a tool? No
AI: It looks like you're interested in Dell laptops within your budget of ₹50000. Based on the observation, I can see that we've got two options that fit your criteria. The DELL Intel Core i5 12th Gen 1235U and the DELL 15 Intel Core i5 13th Gen 1334U are both great choices. If you'd like to know more about these laptops or have any specific questions, feel free to ask!

> Finished chain.
It looks like you're interested in Dell laptops within your budget of ₹50000. Based on the observation, I can see that we've got two options that 

In [28]:
# View history
for msg in memory.load_memory_variables({})["chat_history"]:
    print(f"{msg.type.upper()}: {msg.content}")

# # Or save to a file
# with open("chat_history.txt", "w") as f:
#     for msg in memory.load_memory_variables({})["chat_history"]:
#         f.write(f"{msg.type.upper()}: {msg.content}\n")


HUMAN: Show me Dell laptops under ₹50000
AI: It looks like you're interested in Dell laptops within your budget of ₹50000. Based on the observation, I can see that we've got two options that fit your criteria. The DELL Intel Core i5 12th Gen 1235U and the DELL 15 Intel Core i5 13th Gen 1334U are both great choices. If you'd like to know more about these laptops or have any specific questions, feel free to ask!
HUMAN: Only i5 models please
AI: It looks like we've filtered out some models that don't meet the price requirement of ₹50,000. Since none of the given models exactly match the target price, it might be worth considering laptops that are slightly higher-priced but still within your budget. Would you like me to suggest some options or explore other possibilities?


In [1]:
!pip list

Package                   Version
------------------------- --------------
aiohappyeyeballs          2.6.1
aiohttp                   3.11.18
aiosignal                 1.3.2
altair                    5.5.0
annotated-types           0.7.0
anyio                     4.9.0
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 3.0.0
async-lru                 2.0.5
attrs                     25.3.0
babel                     2.17.0
beautifulsoup4            4.13.3
bleach                    6.2.0
blinker                   1.9.0
cachetools                5.5.2
certifi                   2025.1.31
cffi                      1.17.1
charset-normalizer        3.4.1
click                     8.1.8
colorama                  0.4.6
comm                      0.2.2
contourpy                 1.3.2
cycler                    0.12.1
dataclasses-json          0.6.7
debugpy                   1.8.12
decorator                 5.1.1
defusedxml     